# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [16]:
# setting the stage ;)
import nltk
nltk.download('all')

# Attaching the required data to notebook through google drive
from google.colab import drive
drive.mount('/content/drive')
lines = open('/content/drive/My Drive/holbrook.txt').readlines()
data = [] # data structure to store input data in required format

def dataPrepration(tokens,mode="original"): # utility function for task 1 which converts data into corrrected and original format
    answer = tokens.copy()              # creating a copy of input tokens
    indexes = []
    for i in range(len(tokens)):
        word = tokens[i]
        if '|' in word:                 # only words with | are the ones we need to split and changes for original and corrected mode
            origcorr = word.split('|');
            indexes.append(i)           # store indexes where | occures as these are words which are in original,corrected format
            if mode == "original":
                answer[i] = origcorr[0] # select the left part for original data
            elif mode == "corrected":
                answer[i] = origcorr[1] # select the right part for corrected data
    return answer,indexes

for line in lines:
    tokens = nltk.word_tokenize(line)
    original,indexes = dataPrepration(tokens)                           # get original part of data
    corrected,indexes = dataPrepration(tokens,mode="corrected")         # get corrected part of data
    V = {'original':original,'corrected':corrected,'indexes':indexes}   # create a map containing original,corrected,indexes
    data.append(V)                                                      #append in list for the whole file line by line

assert(data[2] == {
    'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
    'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
    'indexes': [9]
})

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [17]:
test = data[:100]
train = data[100:]

## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [18]:
from collections import Counter

def unigram(word):
    L = [word.lower() for data in train for word in data['corrected']] # create list of all words in lower format present in corrected data
    C = Counter(L)                                                     # create a hashmap using counter of all words gathered in list
    return C[word]                                                     # return count of word given in the corrected dataset

def prob(word):
    L = [word for data in train for word in data['corrected']] # gathered all words present in corrected data set
    return float(unigram(word))/len(L)                         # unigram probablity is count of words by count of total words

# Test your code with the following
assert(unigram("me")==87)

## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [19]:
from nltk.metrics.distance import edit_distance
from collections import defaultdict 

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [20]:
def get_candidates(token):
    S = set(word.lower() for data in train for word in data['corrected']) # set of all unique tokens in train
    D = defaultdict(int) # create a dictionary
    for word in S:
        D[word] = edit_distance(word,token)   # store each word present in train with its edit_distance with given word
    D = sorted(D.items(), key=lambda x: x[1]) # sort the dictionary based on value (edit_distance)
    L = list()
    minEditDistance = D[0][1]                 # minimum edit distance possible between given token and a word from train
    for d in D:
        word = d[0]
        distance = d[1]
        if distance != minEditDistance:
            break
        L.append(word)                        # append all words with lowest edit distance from provided token
    return L                                  # return the list of candidates ( words from train with lowest edit_distance from token )
        
        
# Test your code as follows
print("candidates for word : minde are ",get_candidates("minde"))
#assert get_candidates("minde") == ['mine', 'mind']

candidates for word : minde are  ['mind', 'mine']


## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [21]:
def correct(sentence):
    L=list()
    for word in sentence:
        candidates = get_candidates(word)         # get all possible candidates for word
        D = defaultdict(float)                    # create a dictionary
        for candidate in candidates:
            D[candidate] = prob(candidate)        # store each candidate with there unigram probablity
        D = sorted(D.items(), key=lambda x: x[1]) # sort the dictionary based on value (unigram probablity)
        if bool(D)==True:                         # check dictionary is not empty
            L.append(D[0][0])                     # choose the candidate with lowest unigram probablity and append in list      
    return L

assert(correct(["this","whitr","cat"]) == ['this','white','cat'])   

## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [22]:
def listMatch(X,Y): # utility function to check number of matches between two provided list
    count=0
    for x,y in zip(X,Y):
        if x==y:
            count=count+1
    return count

def accuracy(test):
    totalwords=0    
    correctedwords=0
    for sent in test:
        original = sent['original']
        corrected = sent['corrected']
        indexes = sent['indexes']
        X = correct(original)                                   # send all original words to our spelling checker model
        matches = listMatch(X,corrected)                        # check how many words our model predicted correct
        totalwords += len(original)                             # count all total words in that sentence       
        correctedwords += matches                               # count all words correct in sentence ( this includes all words which were already correct and not modified by our model )
    return float(correctedwords)/float(totalwords)

print(accuracy(test))

0.7209507042253521


## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lectures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)


## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [23]:
"""
In this task I am using a slight modification of interpolation. Rather than just unigram probablity i am using bigram probablity, that too
between (previous word,current word) and (current word,next word).The part where i am using (current word and next word) is my personal modification to formula
as it was giving slightly better result on given data set.
Here rather than just using unigram probablity as a score to find best candidate among all candidates. I am using the following:

  score_of_candidate = alpha*prob(candidate) + (1-alpha)*bigramProb(prev_word,candidate) + (1-alpha)*bigramProb(candidate,next_word) # slighly different from class formula

So adding a refrence of next word and prev word i am scoring a lttle more higher which help us to find better candidates.
"""
totalBigram=list()
for data in train:
    bigrams = [t for t in nltk.bigrams(data['corrected'])] # collect all the bigrams from this line
    totalBigram.extend(bigrams)                            # append thelist of bigram to main list
    
freq= nltk.FreqDist(totalBigram)                           # create a frequency distribution of bigram

def bigramProb(prev_word,word): # utility function to calculate bigram probablity using the frequency distributed generated before
    num,den = freq[(prev_word,word)],0
    for f in freq:
        if f[0] == prev_word:
            den = den + freq[f] # collect all frequency count with all first word as prev_word
    if den==0:
        return 0                # return bigram probablity as zero if the prev_word was not there in train set
    return float(num)/float(den)

def modifiedCorrect(original,corrected,indexes,alpha):
    original_copy=original.copy()         # create a copy of original line
    for i in indexes:
        word = original[i]                # process only the words which are in index list
        candidates = get_candidates(word) 
        prev_word=None                    
        next_word=None                    
        if i!=0:
            prev_word = original[i-1]     # store previous word of candidate if present
        if i!=len(original)-1:
            next_word = original[i+1]     # store next word of candidate if present    
        D = defaultdict(float)            # create a dictionary to store candidates with there score ( score calculated using interpolation formula )
        for candidate in candidates:
            if prev_word!=None:
                bi = bigramProb(prev_word,candidate) 
                D[candidate] = (1-alpha)*bi + alpha*prob(candidate) # if word has prev_word use interpolation formula
            else:
                D[candidate] = alpha*prob(candidate)
                
            if next_word!=None:
                bx = bigramProb(candidate,next_word) 
                D[candidate] += (1-alpha)*bx                        # if word has next_word add using interpolation formula (this formula is little modified from class notes, but its increasing accuracy of given set so used it)
        inverse = [(value, key) for key, value in D.items()]        # sort dictionary based on value (score)
        bestword = max(inverse)[1]                                  # find the word with maximum score
        original_copy[i] = bestword 
    return listMatch(original_copy,corrected)                       # return number of matched words

def accuracy(test,ax):
    totalwords=0    
    correctedwords=0 
    for sent in test:
        original = sent['original']
        corrected = sent['corrected']
        indexes = [i for i in range(len(original))]    # sending all indexes to check ( here it will even check that our model doesnt modify already corrected words )
        matches = modifiedCorrect(original,corrected,indexes,alpha=ax)
        totalwords += len(original)                    # count all total words in that sentence
        correctedwords += matches                      # count all words correct in sentence ( this includes all words which were already correct and not modified by our model )
    return float(correctedwords)/float(totalwords)

print(accuracy(test,0.5)) # 0.5 is alpha for interpolation formula


0.7306338028169014
